## Apiux & SII: Transformacion de contribuyentes con alertas tributarias a contaminados
## ATENCION: proyecto sujeto a mantenimiento continuo. 

## Henry Vega (henrry.vega@api-ux.com)
## Data analyst

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf
import pyspark
import pandas as pd
import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)
from pyspark.sql.types import IntegerType,BooleanType,DateType,StringType

In [2]:
spark = SparkSession.builder \
  .appName("Test")  \
  .config("spark.kerberos.access.hadoopFileSystems","abfs://data@datalakesii.dfs.core.windows.net/") \
  .config("spark.executor.memory", "16g") \
  .config("spark.driver.memory", "12g")\
  .config("spark.executor.cores", "2") \
  .config("spark.driver.maxResultSize", "12g") \
  .getOrCreate()
warnings.filterwarnings('ignore', category=DeprecationWarning)
sc=spark.sparkContext
sc.setLogLevel ('ERROR')
spark.conf.set("spark.sql.parquet.int96RebaseModeInRead", "CORRECTED")
spark.conf.set("spark.sql.parquet.enableVectorizedReader","false")
spark.conf.set("spark.sql.parquet.int96RebaseModeInRead", "CORRECTED")
spark.conf.set("spark.sql.parquet.int96RebaseModeInWrite", "CORRECTED")
spark.conf.set("spark.sql.parquet.datetimeRebaseModeInRead", "CORRECTED")
spark.conf.set("spark.sql.parquet.datetimeRebaseModeInWrite", "CORRECTED")

Setting spark.hadoop.yarn.resourcemanager.principal to yasser.nanjari


In [3]:
#Cargamos la data
a=spark.read.parquet("abfs://data@datalakesii.dfs.core.windows.net/DatosOrigen/DW/DW_TRN_ALERTAS_E")

In [4]:
#Columnas del dataframe
a.printSchema()

root
 |-- ALER_CODIGO_VO: long (nullable = true)
 |-- ALER_COD_TIPO_ALERTA_VO: integer (nullable = true)
 |-- ALER_DESC_TIPO_ALERTA_VO: string (nullable = true)
 |-- ALER_FECHA_ACTIV_VO: timestamp (nullable = true)
 |-- UNOP_UNIDAD_ACTIV: integer (nullable = true)
 |-- ALER_UNIDAD_ACTIV_VO: string (nullable = true)
 |-- ALER_USUARIO_ACTIV_VO: string (nullable = true)
 |-- CONT_RUT: string (nullable = true)
 |-- CONT_DV: string (nullable = true)
 |-- ALER_RUT_VO: string (nullable = true)
 |-- ALER_DV_VO: string (nullable = true)
 |-- UNOP_UNIDAD_DESACTIV: integer (nullable = true)
 |-- ALER_UNIDAD_DESACTIV_VO: string (nullable = true)
 |-- ALER_USUARIO_DESACTIV_VO: string (nullable = true)
 |-- ALER_FECHA_DESACTIV_VO: timestamp (nullable = true)
 |-- ALER_DESCRIPCION_VO: string (nullable = true)
 |-- ALER_FOLIO_BLOQUEO_VO: long (nullable = true)
 |-- ALER_FOLIO_DESBLOQ_VO: long (nullable = true)
 |-- ALER_DESCRIPCION_DESACTIV: string (nullable = true)
 |-- ALER_FECHA_CARGA_DW: timestamp

In [5]:
a.createOrReplaceTempView("alertas")

In [6]:
#Se selecionan las alertas y se contruye la tabla de contaminados
spark.sql("select aler_desc_tipo_alerta_vo,cont_rut from alertas where ALER_COD_TIPO_ALERTA_VO='4110' OR ALER_COD_TIPO_ALERTA_VO='4111' OR ALER_COD_TIPO_ALERTA_VO='4112' OR aler_cod_tipo_alerta_vo='4113' AND aler_fecha_desactiv_vo IS NULL").createOrReplaceTempView("alertas")
spark.sql("SELECT distinct(aler_desc_tipo_alerta_vo)  from alertas").show()
spark.sql("SELECT cont_rut, count(*) as c from alertas group by cont_rut ").createOrReplaceTempView("alertas")
spark.sql("SELECT *  from alertas order by c desc").createOrReplaceTempView("alertas")
spark.sql("SELECT count(*)  from alertas").show()


+------------------------+
|aler_desc_tipo_alerta_vo|
+------------------------+
|    SOCIO Y/O REPRESE...|
|    EMISOR CON ANTECE...|
|    EMISOR Y RECEPTOR...|
|    EMISOR CON ANTECE...|
|    PRESENTA SITUACIO...|
+------------------------+



+--------+
|count(1)|
+--------+
|   32503|
+--------+



Por lo cual, tenemos 25647 contribuyentes contaminados con alguna alerta. Se exportara el dataset con un valor 1 como score para estos contribuyentes. 

In [7]:
out=spark.sql("SELECT cont_rut, 1 as score from alertas").toPandas()
out.reset_index(drop=True).to_csv('/home/cdsw/data/processed/contaminados.csv',index=False)  